In [1]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [2]:
df = pd.read_csv('cleaned_universities.csv')

In [3]:
df.head()

,Country,University,Founded,Type,Enrollment,Link
0,Albania,University of Tirana,1957,Public,35000,https://en.wikipedia.org/wiki/University_of_Ti...
1,Algeria,Constantine University,1978,Public,85000,https://en.wikipedia.org/wiki/List_of_universi...
2,Angola,Agostinho Neto University,1962,Public,29827,https://en.wikipedia.org/wiki/Agostinho_Neto_U...
3,Argentina,University of Buenos Aires,1821,Public,311175,https://en.wikipedia.org/wiki/University_of_Bu...
4,Australia,Monash University,1958,Public,73807,https://en.wikipedia.org/wiki/Monash_University


In [5]:
conn = psycopg2.connect(
    host='localhost',
    port='5434',
    user='amdari_user',
    password='Passw0rd',
    database='amdari_db'
)

In [7]:
cursor = conn.cursor()

In [8]:
create_query_file = open('./sql/create_table.sql')
create_query = create_query_file.read()
create_query

'CREATE TABLE university(\n    id SERIAL PRIMARY KEY,\n    country VARCHAR(255) NOT NULL,\n    name VARCHAR(255) NOT NULL,\n    founded INT NOT NULL,\n    type VARCHAR(255) NOT NULL,\n    enrollment BIGINT NOT NULL,\n    link VARCHAR(255) NOT NULL,\n    constraint unique_combination UNIQUE (country, name)\n)'

In [9]:
cursor.execute(create_query)

DuplicateTable: relation "university" already exists


In [10]:
conn.commit()

In [23]:
conn.rollback()

In [11]:
engine = create_engine('postgresql+psycopg2://', creator=lambda: conn)

In [12]:
pd.read_sql('SELECT * FROM university;', con=engine)

,id,country,name,founded,type,enrollment,link


In [17]:
data = list(df.itertuples(name=None, index=None))
merge_query_file = open('./sql/postgres_upsert.sql')
merge_query = merge_query_file.read()
merge_query

'INSERT INTO university (country,name,founded,type,enrollment,link)\nVALUES (%s, %s, %s, %s, %s, %s)\nON CONFLICT (country, name) DO UPDATE\nSET\n    founded=EXCLUDED.founded,\n    type=EXCLUDED.type,\n    enrollment=EXCLUDED.enrollment,\n    link=EXCLUDED.link;'

In [18]:
cursor.executemany(merge_query, data)

In [19]:
conn.commit()

In [21]:
pd.read_sql('SELECT * FROM university;', con=engine)

,id,country,name,founded,type,enrollment,link
0,70,Albania,University of Tirana,1957,Public,35000,https://en.wikipedia.org/wiki/University_of_Ti...
1,71,Algeria,Constantine University,1978,Public,85000,https://en.wikipedia.org/wiki/List_of_universi...
2,72,Angola,Agostinho Neto University,1962,Public,29827,https://en.wikipedia.org/wiki/Agostinho_Neto_U...
3,73,Argentina,University of Buenos Aires,1821,Public,311175,https://en.wikipedia.org/wiki/University_of_Bu...
4,74,Australia,Monash University,1958,Public,73807,https://en.wikipedia.org/wiki/Monash_University
...,...,...,...,...,...,...,...
64,134,Turkey,Anadolu University,1958,Public,1969733,https://en.wikipedia.org/wiki/Anadolu_University
65,135,United Kingdom,Open University,1969,Public,253075,https://en.wikipedia.org/wiki/Open_University
66,136,United States,Texas A&M University,1876,Public,73284,https://en.wikipedia.org/wiki/Texas_A%26M_Univ...
67,137,Uruguay,University of the Republic,1949,Public,144108,https://en.wikipedia.org/wiki/University_of_th...
